
CURRENTLY:

-takes in SN, EM, NW reconstruction px,py,pz. If nan then converts it to 0 pre scale application

-total of 18 parameters per event 

-outputs 6 bits of data: px,py,pz for nu/nubar



ADDITTION:

-Have added in met_met, met_phi, lep_p1_1 and lep_pt_2



TEST:

-then plotted difference in px and pt from truth for EM, SM, NW and output.

-But here remove events that are not properly reconstructed by the EM, SN, NW models for all



IMPROVMENTS:

-Try doing DNN output of pt, eta, phi? (probably limited difference)?

-add in mu sqaures?

-add in extra kinematics?

-try cutting out useles data before training?

In [1]:
import h5py
import numpy as np
import vector
import awkward as ak
import uproot
import boost_histogram as bh
import mplhep as hep
import matplotlib.pyplot as plt

In [2]:
path = "C:/Users/broad/Documents/Documents/university/summer work/DNN_used/final_using_DNN_folder/"
file = h5py.File(path+'callum_neutrinos.h5', 'r')
mu_squared_eventnumber = np.array(file["eventNumber"])
mu_squared = np.array(file["neutrinos"])
file.close()


In [3]:
treenominal = uproot.open(path+"ttbar_skimmed_july9_v2.root:nominal")

In [4]:
tree = uproot.open(path+"ttbar_skimmed_july9_v2.root:truth")
treerecon = uproot.open(path+"ttbar_skimmed_july9_v2.root:recon")
mask_truth_in_recon = np.isin(tree["eventNumber"].array(),treerecon["eventNumber"].array())
mask_recon_in_truth = np.isin(treerecon["eventNumber"].array(), tree["eventNumber"].array())
mask_nominal_in_truth=mask_recon_in_truth
nu_truth = vector.zip({'pt': tree["nu_pt"].array(), 'eta': tree["nu_eta"].array(), 'phi': tree["nu_phi"].array(), 'm': tree["nu_mass"].array()})[mask_truth_in_recon]
nubar_truth = vector.zip({'pt': tree["nubar_pt"].array(), 'eta': tree["nubar_eta"].array(), 'phi': tree["nubar_phi"].array(), 'm': tree["nubar_mass"].array()})[mask_truth_in_recon]

In [5]:
nu_EM = vector.zip({'pt': treerecon["EM_nu_pt"].array(), 'eta': treerecon["EM_nu_eta"].array(), 'phi': treerecon["EM_nu_phi"].array(), 'm': treerecon["EM_nu_m"].array()})[mask_recon_in_truth]
nubar_EM = vector.zip({'pt': treerecon["EM_nubar_pt"].array(), 'eta': treerecon["EM_nubar_eta"].array(), 'phi': treerecon["EM_nubar_phi"].array(), 'm': treerecon["EM_nubar_m"].array()})[mask_recon_in_truth]
nu_SN = vector.zip({'pt': treerecon["SN_nu_pt"].array(), 'eta': treerecon["SN_nu_eta"].array(), 'phi': treerecon["SN_nu_phi"].array(), 'm': treerecon["SN_nu_m"].array()})[mask_recon_in_truth]
nubar_SN = vector.zip({'pt': treerecon["SN_nubar_pt"].array(), 'eta': treerecon["SN_nubar_eta"].array(), 'phi': treerecon["SN_nubar_phi"].array(), 'm': treerecon["SN_nubar_m"].array()})[mask_recon_in_truth]
nu_NW = vector.zip({'pt': treerecon["NW_nu_pt"].array(), 'eta': treerecon["NW_nu_eta"].array(), 'phi': treerecon["NW_nu_phi"].array(), 'm': treerecon["NW_nu_m"].array()})[mask_recon_in_truth]
nubar_NW = vector.zip({'pt': treerecon["NW_nubar_pt"].array(), 'eta': treerecon["NW_nubar_eta"].array(), 'phi': treerecon["NW_nubar_phi"].array(), 'm': treerecon["NW_nubar_m"].array()})[mask_recon_in_truth]

met_met = treenominal["met_met"].array()
met_phi = treenominal["met_phi"].array()
lep_pt_1  = treenominal["lep_pt"].array()[:,0]
lep_pt_2 = treenominal["lep_pt"].array()[:,1]
lep_eta_1 = treenominal["lep_eta"].array()[:,0]
lep_eta_2 = treenominal["lep_eta"].array()[:,1]
lep_phi_1 = treenominal["lep_phi"].array()[:,0]
lep_phi_2 = treenominal["lep_phi"].array()[:,1]

# #OPTIONAL DONE NOW SO I CAN COMPARE DNN RESULTS TO NEUTRONS!
remove_un_found_events_mask = ((nu_EM.pt>-1) & (nu_EM.pt>-1) & (nu_SN.pt>-1) & (nu_SN.pt>-1) & (nu_NW.pt>-1) & (nu_NW.pt>-1))
nu_EM = nu_EM[remove_un_found_events_mask]
nubar_EM = nubar_EM[remove_un_found_events_mask]
nu_SN = nu_SN[remove_un_found_events_mask]
nubar_SN = nubar_SN[remove_un_found_events_mask]
nu_NW = nu_NW[remove_un_found_events_mask]
nubar_NW = nubar_NW[remove_un_found_events_mask]
met_met =met_met[remove_un_found_events_mask]
met_phi =met_phi[remove_un_found_events_mask]
lep_pt_1 =lep_pt_1[remove_un_found_events_mask]
lep_pt_2 =lep_pt_2[remove_un_found_events_mask]
nu_truth= nu_truth[remove_un_found_events_mask]
nubar_truth=nubar_truth[remove_un_found_events_mask]
lep_eta_1 =lep_eta_1[remove_un_found_events_mask]
lep_eta_2 =lep_eta_2[remove_un_found_events_mask]
lep_phi_1 =lep_phi_1[remove_un_found_events_mask]
lep_phi_2 =lep_phi_2[remove_un_found_events_mask]

In [27]:
import pandas as pd
df = pd.DataFrame(tree["eventNumber"].array()[mask_truth_in_recon][remove_un_found_events_mask], columns=['eventNumber'])
csv_file_path = 'C:/Users/broad/Downloads/neutrino_eventnumber'
df.to_csv(csv_file_path, index=False)


In [34]:
#create dataset with pt, phi, eta for DNN
length=len(nu_EM)/2
length_test=len(nu_EM)-length
h5_file = h5py.File("DNN_train_data_v2.h5", 'w')
group1 = h5_file.create_group('recon')
group1.create_dataset("nu_EM_px", data=nu_EM.px[:length])
group1.create_dataset("nu_EM_py", data=nu_EM.py[:length])
group1.create_dataset("nu_EM_pz", data=nu_EM.pz[:length])
group1.create_dataset("nu_SN_px", data=nu_SN.px[:length])
group1.create_dataset("nu_SN_py", data=nu_SN.py[:length])
group1.create_dataset("nu_SN_pz", data=nu_SN.pz[:length])
group1.create_dataset("nu_NW_px", data=nu_NW.px[:length])
group1.create_dataset("nu_NW_py", data=nu_NW.py[:length])
group1.create_dataset("nu_NW_pz", data=nu_NW.pz[:length])

group1.create_dataset("nubar_EM_px", data=nubar_EM.px[:length])
group1.create_dataset("nubar_EM_py", data=nubar_EM.py[:length])
group1.create_dataset("nubar_EM_pz", data=nubar_EM.pz[:length])
group1.create_dataset("nubar_SN_px", data=nubar_SN.px[:length])
group1.create_dataset("nubar_SN_py", data=nubar_SN.py[:length])
group1.create_dataset("nubar_SN_pz", data=nubar_SN.pz[:length])
group1.create_dataset("nubar_NW_px", data=nubar_NW.px[:length])
group1.create_dataset("nubar_NW_py", data=nubar_NW.py[:length])
group1.create_dataset("nubar_NW_pz", data=nubar_NW.pz[:length])


group1.create_dataset("met_met", data=met_met[:length])
group1.create_dataset("met_phi", data=met_phi[:length])
group1.create_dataset("lep_pt_1", data=lep_pt_1[:length])
group1.create_dataset("lep_pt_2", data=lep_pt_2[:length])
group1.create_dataset("lep_eta_1", data=lep_eta_1[:length])
group1.create_dataset("lep_eta_2", data=lep_eta_2[:length])
group1.create_dataset("lep_phi_1", data=lep_phi_1[:length])
group1.create_dataset("lep_phi_2", data=lep_phi_2[:length])


group2 = h5_file.create_group('truth')
group2.create_dataset("nu_truth_px", data=nu_truth.px[:length])
group2.create_dataset("nu_truth_py", data=nu_truth.py[:length])
group2.create_dataset("nu_truth_pz", data=nu_truth.pz[:length])
group2.create_dataset("nubar_truth_px", data=nubar_truth.px[:length])
group2.create_dataset("nubar_truth_py", data=nubar_truth.py[:length])
group2.create_dataset("nubar_truth_pz", data=nubar_truth.pz[:length])
h5_file.close()

In [35]:
#create dataset with px, py, pz for DNN
length_test=len(nu_EM)-length
h5_file = h5py.File("DNN_test_data_v2.h5", 'w')
group1 = h5_file.create_group('recon')
group1.create_dataset("nu_EM_px", data=nu_EM.px[-length_test:])
group1.create_dataset("nu_EM_py", data=nu_EM.py[-length_test:])
group1.create_dataset("nu_EM_pz", data=nu_EM.pz[-length_test:])
group1.create_dataset("nu_SN_px", data=nu_SN.px[-length_test:])
group1.create_dataset("nu_SN_py", data=nu_SN.py[-length_test:])
group1.create_dataset("nu_SN_pz", data=nu_SN.pz[-length_test:])
group1.create_dataset("nu_NW_px", data=nu_NW.px[-length_test:])
group1.create_dataset("nu_NW_py", data=nu_NW.py[-length_test:])
group1.create_dataset("nu_NW_pz", data=nu_NW.pz[-length_test:])

group1.create_dataset("nubar_EM_px", data=nubar_EM.px[-length_test:])
group1.create_dataset("nubar_EM_py", data=nubar_EM.py[-length_test:])
group1.create_dataset("nubar_EM_pz", data=nubar_EM.pz[-length_test:])
group1.create_dataset("nubar_SN_px", data=nubar_SN.px[-length_test:])
group1.create_dataset("nubar_SN_py", data=nubar_SN.py[-length_test:])
group1.create_dataset("nubar_SN_pz", data=nubar_SN.pz[-length_test:])
group1.create_dataset("nubar_NW_px", data=nubar_NW.px[-length_test:])
group1.create_dataset("nubar_NW_py", data=nubar_NW.py[-length_test:])
group1.create_dataset("nubar_NW_pz", data=nubar_NW.pz[-length_test:])


group1.create_dataset("met_met", data=met_met[-length_test:])
group1.create_dataset("met_phi", data=met_phi[-length_test:])
group1.create_dataset("lep_pt_1", data=lep_pt_1[-length_test:])
group1.create_dataset("lep_pt_2", data=lep_pt_2[-length_test:])
group1.create_dataset("lep_eta_1", data=lep_eta_1[-length_test:])
group1.create_dataset("lep_eta_2", data=lep_eta_2[-length_test:])
group1.create_dataset("lep_phi_1", data=lep_phi_1[-length_test:])
group1.create_dataset("lep_phi_2", data=lep_phi_2[-length_test:])


group2 = h5_file.create_group('truth')
group2.create_dataset("nu_truth_px", data=nu_truth.px[-length_test:])
group2.create_dataset("nu_truth_py", data=nu_truth.py[-length_test:])
group2.create_dataset("nu_truth_pz", data=nu_truth.pz[-length_test:])
group2.create_dataset("nubar_truth_px", data=nubar_truth.px[-length_test:])
group2.create_dataset("nubar_truth_py", data=nubar_truth.py[-length_test:])
group2.create_dataset("nubar_truth_pz", data=nubar_truth.pz[-length_test:])
group2.create_dataset("eventnumber", data=tree["eventNumber"].array()[mask_truth_in_recon][remove_un_found_events_mask][-length_test:])

h5_file.close()

In [33]:
tree["eventNumber"].array()[mask_truth_in_recon][remove_un_found_events_mask][-length_test:]

<Array [1116768, 1116436, ..., 47056749, 47057539] type='770552 * uint64'>